<a href="https://colab.research.google.com/github/JC1335/projet-ccf/blob/master/Final_Projet_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 **Programmation en RDDs (PySpark) - Python**

In [4]:
import time
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark import SparkConf, SparkContext

# Initialisation de SparkSession et du SparkContext
spark = SparkSession.builder.appName("CCF Correct RDD").getOrCreate()
sc = spark.sparkContext

# Accumulateur pour suivre les nouveaux couples
new_pairs_counter = sc.accumulator(0)

def ccf_correct_implementation(edges_rdd, max_iters=20):
    """
    Implémentation correcte de l'algorithme CCF (Connected Component Finder)
    en utilisant les RDD PySpark, suivant la logique du papier académique.
    """
    global new_pairs_counter

    # Étape 1 : Initialiser chaque nœud avec sa propre étiquette de composante
    nodes = edges_rdd.flatMap(lambda edge: [edge[0], edge[1]]).distinct()
    node_component_rdd = nodes.map(lambda node: (node, node))  # (nœud, id_de_composante)

    # Étape 2 : Créer la liste d’adjacence (bidirectionnelle)
    neighbors_rdd = edges_rdd.flatMap(lambda x: [(x[0], x[1]), (x[1], x[0])])

    iteration = 0
    start_time = time.time()

    # Boucle principale : on propage les composantes jusqu’à convergence ou itérations max
    while iteration < max_iters:
        iteration += 1
        print(f" Démarrage de l'itération {iteration}...")

        # Réinitialisation de l'accumulateur pour la nouvelle itération
        new_pairs_counter.value = 0

        # 1️ Joindre chaque nœud avec la composante de ses voisins
        joined_rdd = node_component_rdd.join(neighbors_rdd).map(
            lambda x: (x[1][1], x[1][0])  # (voisin, composante du nœud)
        )

        # 2️ Grouper toutes les composantes possibles pour chaque nœud
        input_for_reducer = joined_rdd.union(node_component_rdd).groupByKey()

        # 3️ Phase de réduction (comme décrite dans la Figure 2 du papier)
        def ccf_iterate_reducer(key_values):
            key, values_iter = key_values
            values = list(values_iter)
            min_val = min(values)

            if min_val < key:
                # Ce nœud peut adopter une composante plus petite : on propage
                yield (key, min_val)
                for val in values:
                    if val != min_val:
                        new_pairs_counter.add(1)
                        yield (val, min_val)
            else:
                # Sinon, il garde sa propre étiquette
                yield (key, key)

        # Application du reducer et dédoublonnage
        ccf_iterate_output = input_for_reducer.flatMap(ccf_iterate_reducer)
        dedup_output = ccf_iterate_output.distinct()

        # Mise à jour de la RDD des composantes
        node_component_rdd = dedup_output

        # Action obligatoire pour forcer l’évaluation du compteur (Spark est paresseux)
        node_component_rdd.count()

        # Si aucune propagation n’a eu lieu, on considère qu’on a convergé
        if new_pairs_counter.value == 0:
            print(f"✅ Convergence atteinte en {iteration} itérations.")
            break

    exec_time = time.time() - start_time
    return node_component_rdd, iteration, exec_time

# Chargement des fichiers et exécution du CCF pour chacun ---
schema = StructType([
    StructField("source", IntegerType(), True),
    StructField("target", IntegerType(), True)
])

# Liste des fichiers à analyser
files = [
    ("G1_1k.csv", "G1"),
    ("G2_5k.csv", "G2"),
    ("G3_8k.csv", "G3"),
    ("G4_10k.csv", "G4")
]

# Liste pour stocker les résultats
results = []

for filename, label in files:
    filepath = f"data/{filename}"  # Etre sûr que le fichier est bien dans le dossier "data"
    print(f"📎 Traitement de {label} ({filepath})...")

    try:
        # Chargement du fichier CSV en DataFrame structuré
        df = spark.read.csv(filepath, header=True, schema=schema)

        # Conversion en RDD de paires (source, target)
        edges_rdd = df.rdd.map(lambda row: (row['source'], row['target']))

        # Lancement de l'algorithme CCF
        components, num_iters, exec_time = ccf_correct_implementation(edges_rdd, max_iters=20)

        # 📈 Statistiques du graphe
        nb_nodes = edges_rdd.flatMap(lambda edge: [edge[0], edge[1]]).distinct().count()
        nb_edges = edges_rdd.count()

        print(f"📊 Données du graphe: {nb_nodes} nœuds, {nb_edges} arêtes")
        print(f"🔁 Itérations : {num_iters}")
        print(f"⏱️ Temps : {round(exec_time, 3)} secondes")
        print("-" * 40)

        # Ajout aux résultats
        results.append((label, nb_nodes, nb_edges, num_iters, round(exec_time, 3)))

    except Exception as e:
        print(f"❌ Erreur avec {label} : {e}")
        print("-" * 40)

# Affichage final dans un tableau Pandas
result_rdd_df = pd.DataFrame(
    results,
    columns=["Graphe", "Nœuds", "Arêtes", "Itérations", "Temps (s)"]
)

print("✅ Résumé des performances (RDD) :")
print(result_rdd_df)


📎 Traitement de G1 (data/G1_1k.csv)...
 Démarrage de l'itération 1...
 Démarrage de l'itération 2...
 Démarrage de l'itération 3...
 Démarrage de l'itération 4...
 Démarrage de l'itération 5...
✅ Convergence atteinte en 5 itérations.
📊 Données du graphe: 996 nœuds, 3000 arêtes
🔁 Itérations : 5
⏱️ Temps : 82.175 secondes
----------------------------------------
📎 Traitement de G2 (data/G2_5k.csv)...
 Démarrage de l'itération 1...
 Démarrage de l'itération 2...
 Démarrage de l'itération 3...
 Démarrage de l'itération 4...
 Démarrage de l'itération 5...
 Démarrage de l'itération 6...
✅ Convergence atteinte en 6 itérations.
📊 Données du graphe: 4983 nœuds, 15000 arêtes
🔁 Itérations : 6
⏱️ Temps : 150.759 secondes
----------------------------------------
📎 Traitement de G3 (data/G3_8k.csv)...
 Démarrage de l'itération 1...
 Démarrage de l'itération 2...
 Démarrage de l'itération 3...
 Démarrage de l'itération 4...
 Démarrage de l'itération 5...
 Démarrage de l'itération 6...
✅ Convergence a

# **2 	Implémentation CCF avec DataFrames _ Python**

In [ ]:
import time
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, least
from pyspark.sql.types import StructType, StructField, IntegerType

# Création de la session Spark
spark = SparkSession.builder \
    .appName("CCF DataFrame Correct") \
    .getOrCreate()

def ccf_dataframe_implementation(edges_df, max_iters=20):
    """
    Implémentation de l’algorithme Connected Component Finder (CCF)
    en utilisant l'API DataFrame de PySpark.

    Objectif : identifier les composantes connexes d'un graphe non orienté
    à partir de ses arêtes (edges_df), en propageant les identifiants
    de composantes les plus petits jusqu'à convergence.
    """

   # Étape 1 : Initialisation — chaque nœud reçoit une étiquette égale à son propre ID
    nodes = edges_df.select("source").union(edges_df.select("target")) \
        .distinct() \
        .withColumnRenamed("source", "node")

    labels = nodes.withColumn("component_id", col("node"))

    iteration = 0
    start_time = time.time()

    # On transforme le graphe en liste d'adjacence non orientée (chaque lien est doublé)
    adj_list = edges_df.select("source", "target").union(
        edges_df.select(col("target").alias("source"), col("source").alias("target"))
    )

    # Boucle principale : on propage les plus petits IDs de composantes
    while iteration < max_iters:
        iteration += 1
        print(f"🔁 Démarrage de l'itération {iteration}...")

        # On renomme la colonne pour éviter les conflits lors des jointures
        labels_renamed = labels.withColumnRenamed("component_id", "current_component_id")

        # 1️ Chaque nœud va consulter les étiquettes de ses voisins
        new_labels = adj_list.join(labels_renamed, adj_list.target == labels_renamed.node) \
            .select(
                adj_list.source.alias("node"),
                labels_renamed.current_component_id.alias("neighbor_component_id")
            ) \
            .groupBy("node") \
            .agg({"neighbor_component_id": "min"}) \
            .withColumnRenamed("min(neighbor_component_id)", "propagated_id")

        # 2️ Jointure avec les étiquettes actuelles pour comparer les valeurs
        current_and_new_labels = labels_renamed.join(new_labels, "node", "left_outer")

        # 3️ Mise à jour : on garde l’ID de composante le plus petit entre l’actuel et le propagé
        updated_labels = current_and_new_labels \
            .withColumn(
                "new_label",
                least(col("current_component_id"), col("propagated_id"))
            ) \
            .select(col("node"), col("new_label").alias("component_id"))

        # 4️ Vérification de convergence : on regarde s’il y a encore des changements
        changes = updated_labels \
            .join(labels.withColumnRenamed("component_id", "old_component_id"), "node") \
            .filter(col("component_id") != col("old_component_id")) \
            .count()

        # 🔁 On prépare les labels pour la prochaine itération
        labels = updated_labels

        # Si aucun changement n’a eu lieu, c’est que l’algorithme a convergé
        if changes == 0:
            print(f"✅ Convergence atteinte en {iteration} itérations.")
            break

    exec_time = time.time() - start_time
    return labels, iteration, exec_time


# Définition du schéma des fichiers CSV
schema = StructType([
    StructField("source", IntegerType(), True),
    StructField("target", IntegerType(), True)
])

# Liste des graphes à traiter
files = [
    ("G1_1k.csv", "G1"),
    ("G2_5k.csv", "G2"),
    ("G3_8k.csv", "G3"),
    ("G4_10k.csv", "G4")
]

# Liste pour stocker les résultats de chaque traitement
results = []

# Traitement de chaque fichier un par un
for filename, label in files:
    filepath = f"data/{filename}"  # Les fichiers doivent être placés dans un dossier 'data'
    print(f"📎 Traitement de {label} ({filepath})...")

    try:
        # Chargement du fichier CSV sous forme de DataFrame
        edges_df = spark.read.csv(filepath, header=True, schema=schema)

        # Exécution de l'algorithme CCF
        components_df, num_iters, exec_time = ccf_dataframe_implementation(edges_df, max_iters=20)

        # Calcul du nombre de nœuds et d’arêtes dans le graphe
        nb_nodes = edges_df.select("source").union(edges_df.select("target")).distinct().count()
        nb_edges = edges_df.count()

        print(f"📊 Données du graphe : {nb_nodes} nœuds, {nb_edges} arêtes")
        print(f"🔁 Itérations : {num_iters}")
        print(f"⏱️ Temps d'exécution : {round(exec_time, 3)} secondes")
        print("-" * 40)

        # Stockage des résultats
        results.append((label, nb_nodes, nb_edges, num_iters, round(exec_time, 3)))

    except Exception as e:
        print(f"❌ Erreur lors du traitement de {label} : {e}")
        print("-" * 40)

# Affichage final des performances sous forme de tableau Pandas
result_df = pd.DataFrame(
    results,
    columns=["Graphe", "Nœuds", "Arêtes", "Itérations", "Temps (s)"]
)

print("✅ Résumé des performances (DataFrame) :")
print(result_df)


📎 Traitement de G1 (data/G1_1k.csv)...
🔁 Démarrage de l'itération 1...
🔁 Démarrage de l'itération 2...
🔁 Démarrage de l'itération 3...
🔁 Démarrage de l'itération 4...
🔁 Démarrage de l'itération 5...
🔁 Démarrage de l'itération 6...
🔁 Démarrage de l'itération 7...
🔁 Démarrage de l'itération 8...
✅ Convergence atteinte en 8 itérations.
📊 Données du graphe : 996 nœuds, 3000 arêtes
🔁 Itérations : 8
⏱️ Temps d'exécution : 140.183 secondes
----------------------------------------
📎 Traitement de G2 (data/G2_5k.csv)...
🔁 Démarrage de l'itération 1...
🔁 Démarrage de l'itération 2...
